<a href="https://colab.research.google.com/github/PranavTyagi00/linear-regression/blob/master/Final_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import library
import os
import io
import json
import torch
from torchvision import transforms
from PIL import Image
import glob
import pandas as pd

In [ ]:
!pip install -q efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_dir   = "/content/drive/My Drive/Colab Notebooks/"
model_path = os.path.join(root_dir, "roof_model.pth")
label_map  = os.path.join(root_dir, "label_map.json")
device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open(label_map, "r") as f:
    label_to_index = json.load(f)
idx_to_name  = {v:k for k,v in label_to_index.items()}

In [ ]:
# Recreate EfficientNet-B3 classifier
n_classes = len(label_to_index)
model = EfficientNet.from_name("efficientnet-b3", num_classes=n_classes)

In [ ]:
# Load the saved weights
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# Move to GPU/CPU and set to eval mode
model = model.to(device)
_ = model.eval()

In [ ]:
infer_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [ ]:
def classify_roof(image_bytes: bytes) -> str:
    """Return predicted roof type"""
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    tensor = infer_transforms(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(tensor)
    idx = logits.argmax(dim=1).item()
    return idx_to_name[idx]

# **Model Testing**

In [ ]:
root_dir    = "/content/drive/My Drive/Colab Notebooks"
test_dir    = os.path.join(root_dir, "test_images")
image_paths = sorted(glob.glob(os.path.join(test_dir, "*.png")))[:10]

In [ ]:
results = []
for path in image_paths:
    with open(path, "rb") as f:
        img_bytes = f.read()
    pred = classify_roof(img_bytes)
    results.append({
        "filename": os.path.basename(path),
        "prediction": pred
    })


In [ ]:
df = pd.DataFrame(results)
print(df)

                           filename    prediction
0  Screenshot 2025-05-01 015834.png      Hip Roof
1  Screenshot 2025-05-01 015917.png     Flat Roof
2  Screenshot 2025-05-01 015952.png      Hip Roof
3  Screenshot 2025-05-01 020012.png     Flat Roof
4  Screenshot 2025-05-01 020031.png  Complex Roof
5  Screenshot 2025-05-01 020051.png      Hip Roof
6  Screenshot 2025-05-01 020144.png  Complex Roof
7  Screenshot 2025-05-01 020215.png  Complex Roof
8  Screenshot 2025-05-01 020226.png     Flat Roof
9  Screenshot 2025-05-01 020243.png  Complex Roof
